## **Introduction**

This module implements an **Abstract Execution (AE)** framework using the `pysvf` library, which is designed to track and reason about symbolic program states during static analysis. At its core, the AE framework provides a high-level interface (`AEState`) for manipulating abstract memory states, including symbolic variables, memory addresses, and abstract values such as intervals and pointers. It is built on top of the lower-level `AbstractState` provided by `pysvf` and supports key operations like loading, storing, joining, widening, narrowing, and state comparison.

To orchestrate symbolic execution across multiple variables and memory objects, the `AbstractExecutionMgr` manages the mapping between variable names and symbolic IDs, provides virtual address generation, and wraps around the abstract state to enable higher-level reasoning and debugging support. It also includes assertions for verifying symbolic properties during analysis.

---

## **Your Task**

You are now tasked with completing a series of additional symbolic tests, from `test1()` through `test8()`. Each test should explore and validate different features of the AE framework such as:

- Symbolic pointers and dereferencing,  
- Modeling memory with offsets (GEP),  
- `malloc` with multiple elements,  
- Symbolic assertions with intervals,  
- Pointer aliasing scenarios,  
- Widening and narrowing state transitions,  
- Complex pointer chains and nested dereferences.  

The goal is to systematically demonstrate the AE framework’s capability to symbolically simulate memory, track pointer relationships, and reason about value ranges and symbolic constraints.

Begin by modeling the scenario for each test using symbolic IDs and virtual memory, then perform relevant store/load operations. Finally, use the `print_abstract_state()` method to inspect and verify the symbolic state at the end of each test.


In [81]:
import pysvf
from pysvf import IntervalValue, AddressValue, AbstractValue, AbstractState, BoundedInt
import sys
from typing import Optional, Union
import copy
class AEState:
    """
    Abstract Execution State
    """
    def __init__(self, state: Optional[AbstractState] = None):
        """
        AEState can be initialized empty or with an existing AbstractState.
        """
        assert isinstance(state, AbstractState) or state is None, "state is not a valid AbstractState object, the type of state is {}".format(type(state))
        if state is not None:
            self.state = state
        else:
            self.state = AbstractState()

    def load_value(self, varId: int) -> AbstractValue:
        """
        Load value from the given variable ID.
        """
        assert isinstance(varId, int), "varId is not a valid integer, the type of varId is {}".format(type(varId))
        res = AbstractValue()
        # TODO: make AddressValue iterable
        for addr in self.state[varId].get_addrs():
            res.join_with(self.state.load(addr))
            break
        return res


    def store_value(self, varId:int , value: Union[AbstractValue, IntervalValue, AddressValue]) -> None:
        """
        Store the given AbstractValue `value` to all memory addresses
        pointed to by the variable with ID `varId`.
        Equivalent to *p = q in C++.
        """
        assert isinstance(varId, int), "varId is not a valid integer, the type of varId is {}".format(type(varId))
        assert isinstance(value, AbstractValue) or isinstance(value, IntervalValue) or isinstance(value, AddressValue), "value is not a valid AbstractValue, IntervalValue, or AddressValue object, the type of value is {}".format(type(value))
        for addr in self.state[varId].get_addrs():# 可选检查
            if isinstance(value, IntervalValue):
                value = AbstractValue(value)
            elif isinstance(value, AddressValue):
                value = AbstractValue(value)
            self.state.store(addr, value)

    def widening(self, other: 'AEState') -> 'AEState':
        """
        Perform a widening operation between this state and `other`,
        returning a new AEState that is the widened result.
        """
        assert isinstance(other, AEState), "other is not a valid AEState object, the type of other is {}".format(type(other))
        widened = self.state.widening(other.state)
        return AEState(widened)

    def narrowing(self, other: 'AEState') -> 'AEState':
        """
        Perform a narrowing operation between this state and `other`,
        returning a new AEState that is the narrowed result.
        """
        assert isinstance(other, AEState), "other is not a valid AEState object, the type of other is {}".format(type(other))
        narrowed = self.state.narrowing(other.state)
        return AEState(narrowed)

    def join_with(self, other: 'AEState') -> None:
        """
        Join this state with `other`, modifying this state in place.
        """
        assert isinstance(other, AEState), "other is not a valid AEState object, the type of other is {}".format(type(other))
        self.state.join_with(other.state)

    def meet_with(self, other: 'AEState') -> None:
        """
        Meet this state with `other`, modifying this state in place.
        """
        assert isinstance(other, AEState), "other is not a valid AEState object, the type of other is {}".format(type(other))
        self.state.meet_with(other.state)

    def __getitem__(self, item: int) -> AbstractValue:
        """
        Get the value associated with the given variable ID.
        """
        assert isinstance(item, int), "item is not a valid integer, the type of item is {}".format(type(item))
        return self.state[item]

    def __setitem__(self, key, value):
        """
        Set the value for the given variable ID.
        """
        assert isinstance(key, int), "key is not a valid integer, the type of key is {}".format(type(key))
        assert isinstance(value, AbstractValue) or isinstance(value, IntervalValue) or isinstance(value, AddressValue), "value is not a valid AbstractValue, IntervalValue, or AddressValue object, the type of value is {}".format(type(value))
        self.state[key] = value

    def print_abstract_state(self) -> None:
        """
        Print the abstract state.
        """
        self.state.print()

    def equals(self, other: 'AEState') -> bool:
        """
        Check if this state is equal to `other`.
        """
        assert isinstance(other, AEState), "other is not a valid AEState object, the type of other is {}".format(type(other))
        return self.state.equals(other.state)

    def clone(self) -> 'AEState':
        """
        Clone this state.
        """
        return AEState(self.state.clone())

class AbstractExecutionMgr:
    """
    Abstract Execution Manager
    """
    def __init__(self):
        self.state = AEState()
        self.current_expr_idx = 1
        self.str_to_id = {} # str: int
        self.id_to_str = {} # int: str


        self.AddressMask = 0x7f000000
        self.FlippedAddressMask = (self.AddressMask^0xffffffff)


    def get_state(self) -> AEState:
        """
        Get the current abstract execution state.
        """
        return self.state

    def set_state(self, state: AEState) -> None:
        """
        Set the abstract execution state.
        """
        assert isinstance(state, AEState), "state is not a valid AEState object, the type of state is {}".format(type(state))
        self.state = state

    def get_internal_id(self, addr: int) -> int:
        """
        Get the internal ID for the given variable name.
        """
        assert isinstance(addr, int), "addr is not a valid integer, the type of addr is {}".format(type(addr))
        return addr & self.FlippedAddressMask

    def get_node_id(self, name: str, size: int = 1) -> int:
        """
        Get the node ID for the given variable name.
        If size > 1, reserves multiple IDs for memory objects (e.g., arrays).
        """
        assert isinstance(name, str), "name is not a valid string, the type of name is {}".format(type(name))
        assert isinstance(size, int), "size is not a valid integer, the type of size is {}".format(type(size))
        if name in self.str_to_id:
            return self.str_to_id[name]
        else:
            self.str_to_id[name] = self.current_expr_idx
            self.id_to_str[self.current_expr_idx] = name
            self.current_expr_idx += size
            return self.str_to_id[name]


    def get_virtual_addr(self, id: int) -> int:
        """
        Get the virtual address for the given variable ID.
        """
        assert isinstance(id, int), "id is not a valid integer, the type of id is {}".format(type(id))
        return self.AddressMask + id

    def get_mem_obj_address(self, name: str) -> int:
        """
        Get the memory object address for the given variable name.
        """
        assert isinstance(name, str), "name is not a valid string, the type of name is {}".format(type(name))
        if name not in self.str_to_id:
            assert False, ""
            sys.exit(1)
        else:
            res = self.str_to_id[name]
            return self.AddressMask + res

    def get_gep_obj_address(self, arr_name: str, offset: int) -> int:
        """
        Get the GEP (GetElementPtr) object address for the given array name and offset.
        """
        assert isinstance(arr_name, str), "arr_name is not a valid string, the type of arr_name is {}".format(type(arr_name))
        assert isinstance(offset, int), "offset is not a valid integer, the type of offset is {}".format(type(offset))
        if arr_name not in self.str_to_id:
            assert False, "Gep BaseObject expr not found?"
            sys.exit(1)

        base_obj_id = self.str_to_id[arr_name]
        gep_obj = base_obj_id + offset

        if base_obj_id == gep_obj:
            return self.AddressMask + base_obj_id
        else:
            return self.AddressMask + gep_obj

    def svf_assert(self, absv: AbstractValue) -> bool:
        """
        Assert the given AbstractValue.
        """
        assert isinstance(absv, AbstractValue), "absv is not a valid AbstractValue object, the type of absv is {}".format(type(absv))
        iv = absv.get_interval()
        if iv.is_numeral():
            if iv.get_numeral() == 0:
                print("\t FAILURE :", "the assertion is unsatisfiable")
                assert False
            else:
                print("\t SUCCESS :", "the assertion is successfully verified")
                return True
        else:
            print("\t FAILURE :", "the assertion is unsatisfiable")
            assert False

    def reset(self):
        """
        Reset the abstract execution manager.
        """
        assert isinstance(self, AbstractExecutionMgr), "self is not a valid AbstractExecutionMgr object, the type of self is {}".format(type(self))
        self.state.state.clear()
        self.current_expr_idx = 1
        self.str_to_id.clear()
        self.id_to_str.clear()

    def print_abstract_state(self) -> None:
        """
        Print the abstract state.
        """
        assert isinstance(self, AbstractExecutionMgr), "self is not a valid AbstractExecutionMgr object, the type of self is {}".format(type(self))
        print("-----------Var and Value-----------")
        field_width = 20

        for var_id, abs_val in self.state.get_var_to_vals():
            var_name = self.id_to_str.get(var_id, "")
            var_info = f"Var{var_id} ({var_name})"
            print(f"{var_info:<{field_width}}: ", end="")

            if abs_val.is_interval():
                print(f"Value: {abs_val.get_interval().to_string()}")
            elif abs_val.is_addr():
                addrs = abs_val.get_addrs()
                addr_str = ", ".join(f"0x{addr:x}" for addr in addrs)
                print(f"Value: {{{addr_str}}}")
            else:
                print("Value: ⊥")

        for addr, abs_val in self.state.get_addr_to_vals():
            addr_str = f"0x{self.get_virtual_addr(addr):x}"
            print(f"{addr_str:<{field_width}}: ", end="")

            if abs_val.is_interval():
                print(f"Value: {abs_val.get_interval().to_string()}")
            elif abs_val.is_addr():
                addrs = abs_val.get_addrs()
                addr_str = ", ".join(f"0x{addr:x}" for addr in addrs)
                print(f"Value: {{{addr_str}}}")
            else:
                print("Value: ⊥")

        print("-----------------------------------------")

As an illustration, the provided `test0()` function walks through a small symbolic program:

- It declares pointers and integer variables,  
- Assigns symbolic memory to `p` using `malloc`,  
- Assigns a concrete integer value to `q`,  
- Stores `q` through the pointer `p`, and  
- Loads the stored value back into `x`.  

This test validates the basic load/store behavior and shows how symbolic memory is tracked through the AE interface.


In [82]:
'''
// A simple example

    int main() {
        int* p;
        int q;
        int* r;
        int x;

        p = malloc();
        q = 5;
        *p = q;
        x = *p;
        assert(x==5);
    }
*/
'''
def test0():
    mgr = AbstractExecutionMgr()
    as_ = mgr.state

    # int** p;
    p = mgr.get_node_id("p")

    # int q;
    q = mgr.get_node_id("q")

    # int* r;
    r = mgr.get_node_id("r")

    # int x;
    x = mgr.get_node_id("x")

    # p = malloc(..);
    malloc = mgr.get_node_id("malloc")
    as_[p] = AddressValue(mgr.get_mem_obj_address("malloc"))

    # q = 5;
    as_[q] = IntervalValue(5, 5)

    # *p = q;
    as_.store_value(p, as_[q])
    
    

    # x = *p;
    as_[x] = as_.load_value(p)

    as_.print_abstract_state()
    

    # assert(x==5)
    # svf_assert(as[x].getInterval() == IntervalValue(5,5))
    res = (as_[mgr.get_node_id("x")].get_interval().eq_interval(IntervalValue(5, 5)))
    assert mgr.svf_assert(AbstractValue(res))
    print("test0 passed")
test0()

	 SUCCESS : the assertion is successfully verified
test0 passed


---

## 🚀 **Your Challenge Starts Here!**

Now that you've seen how the Abstract Execution (AE) framework works in `test0()`, it's time to take on a more exciting challenge.

You are provided with C source code for symbolic execution tests `test1()` through `test8()`. Each test is designed to explore unique aspects of symbolic memory modeling, such as pointer arithmetic, aliasing, control flow analysis, and abstract interpretation techniques like widening and narrowing.

### 🔧 What You Need to Do:

- Complete the corresponding **AE Python implementation** for each test, following the style demonstrated in `test0()`.
- Use symbolic variables, memory allocation (`malloc`), pointer dereferencing, and interval constraints to model the C logic precisely.
- Use `print_abstract_state()` to debug and verify your memory state.
- Make use of `svf_assert()` where applicable to ensure assertions in the original C code hold true under symbolic evaluation.

Each test presents a different scenario that will deepen your understanding of how abstract interpretation works with symbolic memory.

---

🧠 **Tip:** Think of each pointer as a symbolic handle to memory. Track how values move across memory through pointer assignments and dereferencing.

Let’s dive in and begin with `test1()`!

---


In [83]:
'''
// C sourc
e code
int main() {
    int a;
    int b;
    a = 0;
    b = a + 1;
    assert(b > 0);
}
'''
def test1():
    #DO NOT change the following code
    mgr = AbstractExecutionMgr()
    as_ = mgr.state

    # int a;
    a = mgr.get_node_id("a")

    # int b;
    b = mgr.get_node_id("b")
    
    #TODO:implement your code here



    #DO NOT change the following code
    res = (as_[b].get_interval() > IntervalValue(0, 0))
    assert mgr.svf_assert(AbstractValue(res)), "test1 failed"
    print("test1 passed")
test1()

	 FAILURE : the assertion is unsatisfiable


AssertionError: 

In [84]:
'''
// C source code
int main() {
    int* p;
    int q;
    p = malloc;
    *p = 0;
    q = *p;
    *p = 3;
    int b = *p + 1;
    assert(b > 3);
}
'''
def test2():
    #DO NOT change the following code
    mgr = AbstractExecutionMgr()
    as_ = mgr.state
    # int* p;
    p = mgr.get_node_id("p")
    # int q;
    q = mgr.get_node_id("q")
    # p = malloc;
    malloc = mgr.get_node_id("malloc")
    b = mgr.get_node_id("b")
    
    #TODO:implement your code here


    # DO NOT change the following code
    res = (as_[b].get_interval() > IntervalValue(3, 3))
    assert mgr.svf_assert(AbstractValue(res)), "test2 failed"
    print("test2 passed")
test2()

	 FAILURE : the assertion is unsatisfiable


AssertionError: 

In [85]:

'''
// C source code
int main() {
    int** p;
    int* q;
    int* r;
    int x;
    p = malloc1;
    q = malloc2;
    *p = q;
    *q = 10;
    r = *p;
    x = *r;
    assert(x == 10);
}
'''
def test3():
    #DO NOT change the following code
    mgr = AbstractExecutionMgr()
    as_ = mgr.state

    # int** p;
    p = mgr.get_node_id("p")

    # int* q;
    q = mgr.get_node_id("q")

    # int* r;
    r = mgr.get_node_id("r")

    # int x;
    x = mgr.get_node_id("x")

    # p = malloc1;
    malloc1 = mgr.get_node_id("malloc1")
    

    # q = malloc2;
    malloc2 = mgr.get_node_id("malloc2")
    
    #TODO:implement your code here

   

    # DO NOT change the following code
    res = (as_[x].get_interval().eq_interval(IntervalValue(10, 10)))
    assert mgr.svf_assert(AbstractValue(res)), "test3 failed"
    print("test3 passed")
test3()

	 FAILURE : the assertion is unsatisfiable


AssertionError: 

In [86]:

'''
// C source code
int main() {
    int* p;
    int* x;
    int* y;
    int a;
    int b;
    p = malloc;
    x = &p[0];
    y = &p[1];
    *x = 10;
    *y = 11;
    a = *x;
    b = *y;
    assert((a + b) > 20);
}
'''
def test4():
    #DO NOT change the following code
    mgr = AbstractExecutionMgr()
    as_ = mgr.state

    # int* p;
    p = mgr.get_node_id("p")

    # int* x;
    x = mgr.get_node_id("x")

    # int* y;
    y = mgr.get_node_id("y")

    # int a;
    a = mgr.get_node_id("a")

    # int b;
    b = mgr.get_node_id("b")

    # p = malloc;
    malloc = mgr.get_node_id("malloc")
    
    #TODO:implement your code here
   

    # DO NOT change the following code
    res = (as_[a].get_interval() + as_[b].get_interval() > IntervalValue(20, 20))
    assert mgr.svf_assert(AbstractValue(res)), "test4 failed"
    print("test4 passed")
test4()

	 FAILURE : the assertion is unsatisfiable


AssertionError: 

In [87]:

'''
// C source code
    struct A
    { 
        int f0; 
        int* f1;
    };
    int main() {
       struct A* p;
       int* x;
       int* q;
       int** r;
       int* y;
       int z;
    
       p = malloc1;
       x = malloc2;
       *x = 5;
       q = &(p->f0);
       *q = 10;
       r = &(p->f1);
       *r = x;
       y = *r;
       z = *q + *y;
       assert(z==15);
    }

'''
def test5():
    #DO NOT change the following code
    mgr = AbstractExecutionMgr()
    as_ = mgr.state

    # int** p;
    p = mgr.get_node_id("p",2)

    # int* x;
    x = mgr.get_node_id("x")

    # int* q;
    q = mgr.get_node_id("q")

    # int* r;
    r = mgr.get_node_id("r")

    # int* y;
    y = mgr.get_node_id("y")

    # int* z;
    z = mgr.get_node_id("z")

    # p = malloc1;
    malloc1 = mgr.get_node_id("malloc1")
    malloc2 = mgr.get_node_id("malloc2")
    
    #TODO:implement your code here

    # DO NOT change the following code
    res = (as_[mgr.get_node_id("z")].get_interval().eq_interval(IntervalValue(15, 15)))
    assert mgr.svf_assert(AbstractValue(res)), "test5 failed"
    print("test5 passed")
test5()

	 FAILURE : the assertion is unsatisfiable


AssertionError: 

In [88]:

'''
// C source code
    int main(int argv) {  // argv is an interval [4, 10]
        int a;
        int b;
        a = argv + 1;
        b = 5;
        if (a > 10)
            b = a;
        assert(b >= 5);
    }
'''
def test6():
    # DO NOT change the following code
    mgr = AbstractExecutionMgr()
    as_ = mgr.state
    a = mgr.get_node_id("a")
    b = mgr.get_node_id("b")
    argv = mgr.get_node_id("argv")
    
    #TODO: implement your code here
    


    # DO NOT change the following code
    new_as = mgr.state
    res = (new_as[mgr.get_node_id("b")].get_interval() >= IntervalValue(5, 5))
    assert mgr.svf_assert(AbstractValue(res)), "test6 failed"
    print("test6 passed")
test6()

	 FAILURE : the assertion is unsatisfiable


AssertionError: 

In [89]:

'''
// C source code
    int foo(int z) {
        k = z;
        return k;
    }
    int main() {
      int x;
      int y;
      y = foo(2);
      x = foo(3);
      assert(x == 3 && y == 2);
    }

'''
def test7():
    # DO NOT change the following code
    mgr = AbstractExecutionMgr()
    as_ = mgr.state

    x = mgr.get_node_id("x")
    y = mgr.get_node_id("y")
    
    # Call foo(2)
    z = mgr.get_node_id("z")
    
    k = mgr.get_node_id("k")
    
    #TODO: implement your code here
    

    # DO NOT change the following code
    res1 = (as_[mgr.get_node_id("x")].get_interval().eq_interval(IntervalValue(3, 3)))
    res2 = (as_[mgr.get_node_id("y")].get_interval().eq_interval(IntervalValue(2, 2)))
    assert mgr.svf_assert(AbstractValue(res1)), "test7 failed"
    assert mgr.svf_assert(AbstractValue(res2)), "test7 failed"
    print("test7 passed")
test7()

	 FAILURE : the assertion is unsatisfiable


AssertionError: 

In [90]:

'''
// C source code
int main() {
    int x;
    x = 20;
    while (x > 0) {
        x--;
    }
    assert(x == 0);
}
'''
def test8():
    # DO NOT change the following code
    mgr = AbstractExecutionMgr()
    itv = IntervalValue(1, BoundedInt.plus_infinity())
    entry_as = AEState()
    cur_head_as = AEState()
    body_as = AEState()
    exit_as = AEState()
    widen_delay = 3
    x = mgr.get_node_id("x")
    
    
    #TODO: implement your code here
    

    # DO NOT change the following code
    res = (exit_as[x].get_interval().eq_interval(IntervalValue(0, 0)))
    assert mgr.svf_assert(AbstractValue(res)), "test8 failed"
    print("test8 passed")
test8()

	 FAILURE : the assertion is unsatisfiable


AssertionError: 

---

## 🎉 **Congratulations!**

You've successfully navigated through all the symbolic execution challenges from `test1()` to `test8()` using the AE framework!

By completing this lab, you have:

- Modeled symbolic memory using `pysvf`,
- Simulated complex pointer interactions and memory manipulations,
- Applied abstract interpretation techniques such as widening and narrowing,
- Verified program assertions symbolically — without concrete execution!

This journey has equipped you with the tools and techniques to reason about programs at a higher abstraction level. Whether you're analyzing compiler optimizations, detecting bugs, or building static analysis tools, these foundational concepts will serve you well.

Keep exploring, questioning, and building — because every symbolic state starts with a single `malloc()` 😉

---

🌟 Great job, and happy abstract interpreting!
